In [1]:
import os,sys
b_directory = os.path.abspath(os.path.join(os.getcwd(), '..', '..', 'model'))
sys.path.insert(0, b_directory)
from repertoire_cls_bin import *
import numpy as np
import torch
from torch import nn
from torch.optim import Adam
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import (
    average_precision_score,
    roc_auc_score
)

In [2]:
class sx_Dataset(Dataset):
    def __init__(self,data1,data2):
        self.x1 = data1
        self.x2 = data2
        self.len = data1.shape[0]
 
    def __getitem__(self, index):
        return self.x1[index],self.x2[index]
 
    def __len__(self):
        return self.len

In [3]:
def cal_(train_array,train_labels,seed=1,lr=0.0001,EPOCH=50,BATCH_SIZE=32, device='cuda:0',save_path='./model.pt'):
    torch.manual_seed(seed)
    model=classification_model(tcr_dim=train_array.shape[-1],nums=train_array.shape[-2])
    
    model=model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    
    for epoch in range(EPOCH):
        model.train()
        train_dataloader= DataLoader(dataset=sx_Dataset(train_array,train_labels),batch_size=BATCH_SIZE,shuffle=True,num_workers=4,drop_last=True)
        total_loss = 0
        for tra_step, (cdr3,label) in enumerate(train_dataloader):
            cdr3=torch.tensor(cdr3,dtype=torch.float32)
            cdr3=cdr3.to(device)
            
            label=torch.tensor(label,dtype=torch.float32)
            label=label.to(device)
            
            pred = model(cdr3)
            pred = pred.flatten()
            
            loss = F.binary_cross_entropy(pred,label)
            
            optimizer.zero_grad()
            loss.requires_grad_(True)
            loss.backward()
            optimizer.step()
         
    torch.save(model,save_path)

In [4]:
train_cdr3=np.load('../../tmp_data/4/train_cdr3.npy')
train_labels=np.load('../../tmp_data/4/train_labels.npy')
model_save_path='./model.pt'
cuda='cuda:0'

In [5]:
cal_(train_cdr3,train_labels,device=cuda,save_path=model_save_path)

/tmp/ipykernel_217507/2139930225.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  cdr3=torch.tensor(cdr3,dtype=torch.float32)
/tmp/ipykernel_217507/2139930225.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label=torch.tensor(label,dtype=torch.float32)


In [6]:
test_cdr3=np.load('../../tmp_data/4/test_cdr3.npy')
test_labels=np.load('../../tmp_data/4/test_labels.npy')

In [7]:
model=torch.load(model_save_path)

/tmp/ipykernel_217507/3839462922.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model=torch.load(model_save_path)


In [8]:
model.eval()

classification_model(
  (get_beta): get_msg(
    (dropout): Dropout(p=0.5, inplace=False)
    (itm_head_1): Linear(in_features=30720, out_features=1024, bias=True)
    (itm_head_2): Linear(in_features=1024, out_features=256, bias=True)
  )
  (self_attention): MultiLayerSelfAttention(
    (attention_layers): ModuleList(
      (0-7): 8 x SelfAttention(
        (query): Linear(in_features=256, out_features=256, bias=True)
        (key): Linear(in_features=256, out_features=256, bias=True)
        (value): Linear(in_features=256, out_features=256, bias=True)
        (fc_out): Linear(in_features=256, out_features=256, bias=True)
      )
    )
  )
  (dense): Sequential(
    (0): Linear(in_features=256, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=256, out_features=20, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.5, inplace=False)
    (6): Linear(in_features=20, out_features=1, bias=True)
    (7): Sigmoid()
  )
)

In [9]:
preds=model(torch.tensor(test_cdr3,dtype=torch.float32).to(cuda)
           )

In [10]:
roc_auc_score(test_labels,preds.detach().cpu().numpy())

0.9773684210526316